In [1]:
import pandas as pd
import seaborn as sns

from fairpair import *

In [2]:
G = FairPairGraph()
G.generate_groups(15, 5)
G.group_assign_scores(nodes=G.majority_nodes, distr=Distributions.normal_distr)
G.group_assign_scores(nodes=G.minority_nodes, distr=Distributions.normal_distr, loc=0.3, scale=0.2) # give a disadvantage to the minority

In [3]:
sampler = ProbKnockoutSampling(G, warn=False)
sampler.apply(iter=100, k=1)

In [4]:
ranker = RankRecovery(G)
ranking, other_nodes = ranker.apply(rank_using=davidScore) # by default, apply rankCentrality method

In [5]:
ranker._print_with_score(ranking)

,node,orig score,rank score
0,12,0.635905,25.718750
1,4,0.869468,22.341667
2,7,0.516928,12.303571
3,1,0.369742,4.163636
4,3,0.347859,-3.559524
5,6,0.208452,-3.660714
6,0,0.157930,-4.608333
7,13,0.156119,-6.678571
8,8,0.196995,-7.443155
9,14,0.001194,-7.443155


In [6]:
print("Majority NDCG:", ranker.calc_NDCG(G.majority, ranking))
print("Minority NDCG:", ranker.calc_NDCG(G.minority, ranking))

Majority NDCG: 0.9843736058342109
Minority NDCG: 0.9977635867647169


In [7]:
print("Majority MSE:", ranker.calc_MSE(G.majority, ranking))
print("Minority MSE:", ranker.calc_MSE(G.minority, ranking))

Majority MSE: 0.027060044879013673
Minority MSE: 0.04068289265872983
